In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

In [27]:
feature_maps = (torch.randn(3, 256, 216, 184), torch.randn(3, 256, 108, 92), torch.randn(3, 256, 54, 46), torch.randn(3, 256, 27, 23))

In [28]:
rois = torch.Tensor([[  0.0000, 557.9707, 366.0170, 697.4088, 566.9425],
[  1.0000,   0.0000, 622.1016, 143.7075, 709.8333],
[  1.0000,  16.5338, 620.5057, 197.3975, 710.6006],
[  2.0000, 223.7691, 473.1749, 359.9460, 628.8488],
[  2.0000, 208.7294, 522.8946, 341.2042, 643.8867]])

In [29]:
rois_index = rois[:, 0].long()

In [30]:
rois_ = rois[:, 1:]

In [31]:
i = 0
curf = feature_maps[i]
scale_factor = 4 * (i + 1)

In [32]:
rois_rescale = torch.round(rois_ / scale_factor).long()

In [33]:
rois_index

tensor([0, 1, 1, 2, 2])

In [34]:
rois_rescale

tensor([[139,  92, 174, 142],
        [  0, 156,  36, 177],
        [  4, 155,  49, 178],
        [ 56, 118,  90, 157],
        [ 52, 131,  85, 161]])

In [35]:
oneD_coordinate = torch.arange(curf.size(2) * curf.size(3)).reshape(1, curf.size(2), curf.size(3))
y_coordinate = torch.floor(oneD_coordinate / curf.size(3))
x_coordinate = oneD_coordinate % curf.size(3)

In [36]:
oneD_coordinate

tensor([[[    0,     1,     2,  ...,   181,   182,   183],
         [  184,   185,   186,  ...,   365,   366,   367],
         [  368,   369,   370,  ...,   549,   550,   551],
         ...,
         [39192, 39193, 39194,  ..., 39373, 39374, 39375],
         [39376, 39377, 39378,  ..., 39557, 39558, 39559],
         [39560, 39561, 39562,  ..., 39741, 39742, 39743]]])

In [37]:
y_coordinate

tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         ...,
         [213., 213., 213.,  ..., 213., 213., 213.],
         [214., 214., 214.,  ..., 214., 214., 214.],
         [215., 215., 215.,  ..., 215., 215., 215.]]])

In [38]:
x_coordinate

tensor([[[  0,   1,   2,  ..., 181, 182, 183],
         [  0,   1,   2,  ..., 181, 182, 183],
         [  0,   1,   2,  ..., 181, 182, 183],
         ...,
         [  0,   1,   2,  ..., 181, 182, 183],
         [  0,   1,   2,  ..., 181, 182, 183],
         [  0,   1,   2,  ..., 181, 182, 183]]])

In [39]:
y_coordinate = y_coordinate.repeat(rois_rescale.size(0), 1, 1)
x_coordinate = x_coordinate.repeat(rois_rescale.size(0), 1, 1)

In [40]:
y_coordinate.size()

torch.Size([5, 216, 184])

In [41]:
one_mask = (x_coordinate >= rois_rescale[:, 0].reshape(-1, 1, 1)) & \
            (x_coordinate <= rois_rescale[:, 2].reshape(-1, 1, 1)) & \
            (y_coordinate >= rois_rescale[:, 1].reshape(-1, 1, 1)) & \
            (y_coordinate <= rois_rescale[:, 3].reshape(-1, 1, 1))

In [42]:
one_mask = one_mask.reshape(-1, 1, curf.size(2), curf.size(3))
one_mask.size()

torch.Size([5, 1, 216, 184])

In [43]:
feature_for_rois = curf[rois_index]
feature_for_rois.size()

torch.Size([5, 256, 216, 184])

In [44]:
curf.element_size() * curf.nelement()

122093568

In [45]:
# 过大，不能这么做，用其他办法，比如深度换广度
122093568 * 49 / 1073741824

5.57171630859375

In [46]:
one_mask[1][0][92]

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [47]:
masked = feature_for_rois.masked_fill_(~one_mask, 0)

In [48]:
masked[1][0][156]

tensor([-1.0733, -0.0987,  1.0486,  0.3846, -0.2713,  0.6711, -1.6563,  1.7042,
         0.9191, -0.7272,  1.1034,  0.0026, -0.4326,  1.6772,  0.0560,  0.2816,
         0.8218,  0.1034,  0.6547, -0.8347,  0.8229, -0.4328,  0.5896,  0.9136,
        -1.0090, -0.1894, -1.3477,  0.2294, -0.0620, -0.6081, -0.2344, -0.8739,
        -0.7584,  0.8919,  0.8030,  1.6139, -1.2724,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

In [49]:
masked.size()

torch.Size([5, 256, 216, 184])

In [50]:
# 方案二：
# 把x的power的cumsum算出来之后再根据坐标相减
# e.g. 对x1 y1 x2 y2来说
# 他们的和等于的是
# cumsum[x2][y2] - cumsum[x1][y2] - cumsum[x2][y1] + cumsum[x1][y1]

In [51]:
curf = feature_maps[0]
curf.size()

torch.Size([3, 256, 216, 184])

In [52]:
p = nn.Parameter(torch.Tensor([3, 3, 3, 3]))
minimumx = nn.Parameter(torch.Tensor([1e-6]), requires_grad=False)

In [53]:
xpower = torch.pow(torch.maximum(curf, minimumx), p[0])

In [54]:
xpower_cumsum = xpower.cumsum(2).cumsum(3)

In [55]:
xpower_cumsum.size()

torch.Size([3, 256, 216, 184])

In [56]:
x1 = rois_rescale[:, 0]
y1 = rois_rescale[:, 1]
x2 = rois_rescale[:, 2]
y2 = rois_rescale[:, 3]

In [57]:
x1.size()

torch.Size([5])

In [62]:
v1 = xpower_cumsum[rois_index, :, y1, x1]
v2 = xpower_cumsum[rois_index, :, y2, x2]
v3 = xpower_cumsum[rois_index, :, y2, x1]
v4 = xpower_cumsum[rois_index, :, y2, x2]

roi_power_sum = v4 - v3 - v2 + v1

In [63]:
roi_power_sum.size()

torch.Size([5, 256])

In [64]:
x1

tensor([139,   0,   4,  56,  52])

In [65]:
y1

tensor([ 92, 156, 155, 118, 131])

In [66]:
xpower_cumsum[0, 0, 92, 139]

tensor(10322.5391, grad_fn=<SelectBackward0>)

In [67]:
v1[0][0]

tensor(10322.5391, grad_fn=<SelectBackward0>)

In [37]:
origin = curf[rois_index, :, x1, y1]
origin[0][0]

tensor(0.8135)

In [38]:
x1

tensor([139,   0,   4,  56,  52])

In [39]:
y1

tensor([ 92, 156, 155, 118, 131])

In [40]:
w = x2 - x1
h = y2 - y1

In [41]:
winc = torch.ceil(w / 7).long()
hinc = torch.ceil(h / 7).long()

In [42]:
subrois_x1 = []
subrois_x2 = []
subrois_y1 = []
subrois_y2 = []
subrois_area = []
for i in range(7):
    cur_y1 = y1 + hinc * i
    if i < 6:
        cur_y2 = y1 + hinc * (i + 1) - 1
    else:
        cur_y2 = y2
    for j in range(7):
        cur_x1 = x1 + winc * j
        if j < 6:
            cur_x2 = x1 + winc * (j + 1) - 1
        else:
            cur_x2 = x2
            
        subrois_x1.append(cur_x1)
        subrois_y1.append(cur_y1)
        subrois_x2.append(cur_x2)
        subrois_y2.append(cur_y2)
        
        subarea = (cur_x2 - cur_x1 + 1) * (cur_y2 - cur_y1 + 1)
        subrois_area.append(subarea)

In [43]:
sx1 = torch.cat(subrois_x1, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)
sy1 = torch.cat(subrois_y1, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)
sx2 = torch.cat(subrois_x2, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)
sy2 = torch.cat(subrois_y2, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)
ss = torch.cat(subrois_area, dim=-1).reshape(7, 7, 5).permute(2, 1, 0)

In [44]:
rois_index_sub = rois_index.reshape(-1, 1, 1).repeat(1, 7, 7)

In [45]:
v1 = xpower_cumsum[rois_index_sub, :, sx1, sy1]
v2 = xpower_cumsum[rois_index_sub, :, sx1, sy2]
v3 = xpower_cumsum[rois_index_sub, :, sx2, sy1]
v4 = xpower_cumsum[rois_index_sub, :, sx2, sy2]

In [46]:
subroi_power_sum = v4 - v3 - v2 + v1

In [47]:
subroi_power_sum.size()

torch.Size([5, 7, 7, 256])

In [48]:
subroi_mean = subroi_power_sum / ss.unsqueeze(-1)

In [49]:
ss.unsqueeze(-1).size()

torch.Size([5, 7, 7, 1])

In [50]:
proj = nn.ModuleList([nn.Conv2d(256, 256, 1, 1, 0) for _ in range(4)])

In [51]:
gem = torch.pow(subroi_mean, 1.0 / p[0])

In [53]:
feat_level = proj[0](gem.permute(0, 3, 1, 2))

In [ ]:
## test all

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys

In [2]:
def map_roi_levels(rois, num_levels):
        """Map rois to corresponding feature levels by scales.

        - scale < finest_scale * 2: level 0
        - finest_scale * 2 <= scale < finest_scale * 4: level 1
        - finest_scale * 4 <= scale < finest_scale * 8: level 2
        - scale >= finest_scale * 8: level 3

        Args:
            rois (Tensor): Input RoIs, shape (k, 5).
            num_levels (int): Total level number.

        Returns:
            Tensor: Level index (0-based) of each RoI, shape (k, )
        """
        scale = torch.sqrt(
            (rois[:, 3] - rois[:, 1]) * (rois[:, 4] - rois[:, 2]))
        target_lvls = torch.floor(torch.log2(scale / 56 + 1e-6))
        target_lvls = target_lvls.clamp(min=0, max=num_levels - 1).long()
        return target_lvls

In [171]:
# feats = (torch.relu(torch.randn(3, 256, 224, 168)), torch.relu(torch.randn(3, 256, 112, 84)), torch.relu(torch.randn(3, 256, 56, 42)), torch.relu(torch.randn(3, 256, 28, 21)))
feats = (torch.randn(3, 256, 224, 168), torch.randn(3, 256, 112, 84), torch.randn(3, 256, 56, 42), torch.randn(3, 256, 28, 21))
rois = torch.Tensor([[  0.0000,  44.5265, 533.2392, 123.3619, 646.8044],
[  0.0000, 483.3849, 414.3447, 544.0000, 561.0616],
[  0.0000, 464.8555, 361.8051, 544.0000, 513.1793],
[  2.0000, 212.0784, 225.8543, 254.5084, 249.5771],
[  2.0000, 247.9729, 760.3569, 292.2449, 782.3408],
[  2.0000, 370.9329, 319.0714, 461.9206, 359.8063],
[  0.0000,   0.0000, 546.8749, 138.5107, 643.9124],
[  0.0000,  20.2614, 524.2296, 143.1214, 654.5751],
[  0.0000, 338.6020, 213.8744, 470.8668, 356.2858],
[  2.0000, 563.3346, 466.3733, 672.0000, 598.5251],
[  2.0000, 191.0886,  11.1537, 368.5119, 110.5652],
[  2.0000, 147.9487, 314.7806, 279.8951, 450.1862],
[0.0000e+00, 8.1635e+00, 3.1470e+02, 2.9839e+02, 5.9779e+02],
[0.0000e+00, 1.0332e+02, 3.1823e+02, 3.7490e+02, 6.1636e+02],
[0.0000e+00, 0.0000e+00, 2.6797e+02, 2.4527e+02, 5.2831e+02],
[0.0000e+00, 2.6465e+02, 3.3791e+02, 5.4215e+02, 6.2000e+02],
[0.0000e+00, 0.0000e+00, 4.0628e+01, 1.4740e+02, 4.4955e+02],
[0.0000e+00, 3.2377e+02, 2.4155e+02, 5.4400e+02, 5.2082e+02],
[0.0000e+00, 3.2348e+02, 3.1718e+02, 5.4400e+02, 6.0229e+02],
[0.0000e+00, 2.3139e+02, 2.3201e+02, 5.4400e+02, 8.1500e+02],
[0.0000e+00, 3.5330e+02, 4.1191e+02, 5.4400e+02, 6.9351e+02],
[0.0000e+00, 3.3930e+02, 4.8037e+02, 5.4400e+02, 7.5963e+02],
[0.0000e+00, 1.1803e+02, 4.6942e+02, 3.7715e+02, 7.5625e+02],
[0.0000e+00, 6.3730e+01, 3.5782e+02, 3.4386e+02, 6.3828e+02],
[0.0000e+00, 4.6200e+01, 4.2272e+02, 3.1291e+02, 6.9105e+02],
[0.0000e+00, 0.0000e+00, 2.2000e+02, 2.1545e+02, 4.8887e+02],
[0.0000e+00, 2.3805e+02, 0.0000e+00, 5.0801e+02, 2.1661e+02],
[  0.0000,  54.9208, 167.9849, 544.0000, 757.0776],
[  0.0000,   0.0000, 139.9310, 483.0693, 706.0065],
[  0.0000,  83.9238, 311.6150, 544.0000, 815.0000],
[  0.0000,   0.0000,   0.0000, 423.8024, 520.2032],
[  0.0000, 157.6781,   0.0000, 491.9783, 657.8700],
[  0.0000,  93.0500,   0.0000, 441.3786, 755.1307],
[  1.0000,   0.0000,   0.0000, 480.0000, 760.3838],])

In [248]:
p = nn.Parameter(torch.Tensor([3, 3, 3, 3]))
proj = nn.ModuleList([nn.Conv2d(256, 256, 1, 1, 0) for _ in range(4)])
minimumx = nn.Parameter(torch.Tensor([1e-6]), requires_grad=False)
featmap_strides = [4, 8, 16, 32]
rlen = len(rois)

In [249]:
num_levels = len(feats)

In [250]:
target_lvls = map_roi_levels(rois, num_levels)

In [251]:
len(target_lvls)

34

In [263]:
roi_feats_list = []
rois_index = rois[:, 0].long()
rois_index_sub = rois_index.reshape(-1, 1, 1).repeat(1, 7, 7)
for level in range(num_levels):
    curf = feats[level]
    print("curf", curf.size(), curf.isnan().sum())
    print("curf < 0", (curf < 0).sum())
    batch_size, channels, ch, cw = curf.size()
    scale_factor = featmap_strides[level]
    rois_rescale = torch.round(rois[:, 1:] / scale_factor).long()
    print(rois_rescale)
    # gem power
    xpower = torch.pow(torch.maximum(curf, minimumx), p[level])
    print("xpower <= 0", (xpower <= 0).sum(), xpower.size())
#     print("xpower", xpower.size(), xpower.isnan().sum())
    # gem sum, but store as cumsum
    xpower_cumsum = xpower.cumsum(2).cumsum(3)
    print("xpower_cumsum <= 0", (xpower_cumsum <= 0).sum())
#     print("xpower_cumsum", xpower_cumsum.size(), xpower_cumsum.isnan().sum())
    # rescaled coordinates
    x1 = torch.clamp(rois_rescale[:, 0], min=1, max=cw - 1)
    y1 = torch.clamp(rois_rescale[:, 1], min=1, max=ch - 1)
    x2 = torch.clamp(rois_rescale[:, 2], min=1, max=cw - 1)
    y2 = torch.clamp(rois_rescale[:, 3], min=1, max=ch - 1)
#     print("x1", x1.size(), x1.isnan().sum())
#     print("y1", y1.size(), y1.isnan().sum())
#     print("x2", x2.size(), x2.isnan().sum())
#     print("y2", y2.size(), y2.isnan().sum())
    # subregion intervals generate 7 * 7
    w = x2 - x1 + 1
    h = y2 - y1 + 1
    print("w and h", (w <= 0).sum(), (h <= 0).sum())
    winc = torch.floor(w / 7).long()
    hinc = torch.floor(h / 7).long()
    print("winc and hinc", (winc < 0).sum(), (hinc < 0).sum())
#     print("x1", x1.size(), x1.isnan().sum())
#     print("y1", y1.size(), y1.isnan().sum())
#     print("x2", x2.size(), x2.isnan().sum())
#     print("y2", y2.size(), y2.isnan().sum())
    # find all subregion coordinates and area
    subrois_x1 = []
    subrois_x2 = []
    subrois_y1 = []
    subrois_y2 = []
    subrois_area = []
    for i in range(7):
        cur_y1 = y1 + hinc * i
        if i < 6:
            cur_y2 = cur_y1 + hinc
        else:
            cur_y2 = y2
        for j in range(7):
            cur_x1 = x1 + winc * j
            if j < 6:
                cur_x2 = cur_x1 + winc
            else:
                cur_x2 = x2

            subrois_x1.append(cur_x1)
            subrois_y1.append(cur_y1)
            subrois_x2.append(cur_x2)
            subrois_y2.append(cur_y2)
            
            subarea = (cur_x2 - cur_x1 + 1) * (cur_y2 - cur_y1 + 1)
            subrois_area.append(subarea)
    # concat them all together
    # size(N, 7, 7)
    sx1 = torch.cat(subrois_x1, dim=-1).reshape(7, 7, rlen).permute(2, 1, 0)
    sy1 = torch.cat(subrois_y1, dim=-1).reshape(7, 7, rlen).permute(2, 1, 0)
    sx2 = torch.cat(subrois_x2, dim=-1).reshape(7, 7, rlen).permute(2, 1, 0)
    sy2 = torch.cat(subrois_y2, dim=-1).reshape(7, 7, rlen).permute(2, 1, 0)
    ss = torch.cat(subrois_area, dim=-1).reshape(7, 7, rlen).permute(2, 1, 0)
    print("ss", ss.size(), ss.isnan().sum())
    print("ss < 0", (ss < 0).sum())
    # get ROI subregion gem sum
    v1 = xpower_cumsum[rois_index_sub, :, sy1 - 1, sx1 - 1]
    v2 = xpower_cumsum[rois_index_sub, :, sy1 - 1, sx2]
    v3 = xpower_cumsum[rois_index_sub, :, sy2, sx1 - 1]
    v4 = xpower_cumsum[rois_index_sub, :, sy2, sx2]
#     print("v1", v1.size(), v1.isnan().sum())
#     print("v2", v2.size(), v2.isnan().sum())
#     print("v3", v3.size(), v3.isnan().sum())
#     print("v4", v4.size(), v4.isnan().sum())
    # get real sum by inclusive-exclusive algo
    subroi_power_sum = v4 - v3 - v2 + v1
    print("subroi_power_sum", subroi_power_sum.size(), subroi_power_sum.isnan().sum())
    print("subroi_power_sum <= 0", (subroi_power_sum <= 0).sum())
    # calculate mean by divide area
    subroi_mean = subroi_power_sum / (ss.unsqueeze(-1))
    print("subroi_mean", subroi_mean.size(), subroi_mean.isnan().sum())
#     gem = torch.sign(subroi_mean) * torch.pow(torch.abs(subroi_mean), 1.0 / p[level])
    gem = torch.pow(torch.maximum(subroi_mean, minimumx), 1.0 / p[level])
    print("gem", gem.size(), gem.isnan().sum())
    feat_level = proj[level](gem.permute(0, 3, 1, 2))
    roi_feats_list.append(feat_level)


curf torch.Size([3, 256, 224, 168]) tensor(0)
curf < 0 tensor(14449276)
tensor([[ 11, 133,  31, 162],
        [121, 104, 136, 140],
        [116,  90, 136, 128],
        [ 53,  56,  64,  62],
        [ 62, 190,  73, 196],
        [ 93,  80, 115,  90],
        [  0, 137,  35, 161],
        [  5, 131,  36, 164],
        [ 85,  53, 118,  89],
        [141, 117, 168, 150],
        [ 48,   3,  92,  28],
        [ 37,  79,  70, 113],
        [  2,  79,  75, 149],
        [ 26,  80,  94, 154],
        [  0,  67,  61, 132],
        [ 66,  84, 136, 155],
        [  0,  10,  37, 112],
        [ 81,  60, 136, 130],
        [ 81,  79, 136, 151],
        [ 58,  58, 136, 204],
        [ 88, 103, 136, 173],
        [ 85, 120, 136, 190],
        [ 30, 117,  94, 189],
        [ 16,  89,  86, 160],
        [ 12, 106,  78, 173],
        [  0,  55,  54, 122],
        [ 60,   0, 127,  54],
        [ 14,  42, 136, 189],
        [  0,  35, 121, 177],
        [ 21,  78, 136, 204],
        [  0,   0, 106, 130]

In [264]:
weights = nn.Parameter(torch.Tensor([[0.8, 0.4, 0.2, 0.1],
                                    [0.2, 0.8, 0.4, 0.1],
                                    [0.1, 0.2, 0.8, 0.4],
                                    [0.1, 0.2, 0.4, 0.8]]), requires_grad=False)

In [265]:
add_weights = weights[target_lvls]

In [266]:
add_weights.size()

torch.Size([34, 4])

In [267]:
roi_feats_list[0].size()

torch.Size([34, 256, 7, 7])

In [268]:
roi_feats = torch.cat(roi_feats_list, dim=1).reshape(-1, num_levels, 256, 7, 7)

In [269]:
roi_feats.size()

torch.Size([34, 4, 256, 7, 7])

In [270]:
roi_feats_weighted = torch.sum(roi_feats * add_weights.reshape(-1, num_levels, 1, 1, 1),dim=1)

In [271]:
add_weights.reshape(-1, num_levels, 1, 1, 1).size()

torch.Size([34, 4, 1, 1, 1])

In [272]:
roi_feats_weighted.size()

torch.Size([34, 256, 7, 7])

In [273]:
roi_feats_weighted

tensor([[[[-9.1632e-02, -1.3938e-01, -2.4299e-01,  ..., -2.7541e-01,
           -1.0297e-01, -3.3561e-02],
          [-2.9379e-01, -1.7696e-01,  1.1316e-01,  ..., -3.2951e-01,
           -2.7151e-01, -4.0560e-01],
          [-2.8665e-01, -3.8054e-02, -7.9367e-03,  ..., -1.3270e-01,
           -1.6945e-01, -1.5531e-01],
          ...,
          [-1.8094e-01, -1.4640e-01, -1.5507e-01,  ..., -2.2006e-01,
           -3.7539e-01, -3.2098e-01],
          [-3.4718e-01, -2.9138e-01, -2.3738e-01,  ...,  2.4725e-02,
           -1.3747e-01, -1.7044e-01],
          [-2.6693e-01, -2.7101e-01, -2.6784e-01,  ..., -1.2262e-01,
           -4.0956e-02, -2.1274e-01]],

         [[ 1.4241e-01,  1.3577e-01,  2.0397e-01,  ...,  3.7264e-01,
            1.4633e-01,  9.7923e-02],
          [ 2.7038e-01,  8.5220e-02, -3.2925e-02,  ...,  2.0126e-01,
            2.9708e-01,  1.2126e-01],
          [ 3.9203e-01,  2.2669e-01,  1.5097e-01,  ...,  3.3299e-01,
            5.6803e-01,  2.8918e-01],
          ...,
     

In [232]:
xpower_cumsum[0][0]

tensor([[1.0000e-06, 5.6960e+00, 5.7210e+00, 5.7210e+00, 5.7434e+00, 1.3487e+01,
         1.4545e+01, 1.4545e+01, 1.7575e+01, 1.7575e+01, 1.7575e+01, 1.7575e+01,
         1.7575e+01, 1.7591e+01, 1.7591e+01, 1.7612e+01, 1.8178e+01, 1.8178e+01,
         1.8215e+01, 1.8215e+01, 1.8215e+01],
        [2.5110e-02, 5.7211e+00, 5.7461e+00, 7.3705e+00, 7.3928e+00, 1.7309e+01,
         1.8442e+01, 1.8442e+01, 2.1472e+01, 2.3166e+01, 2.3166e+01, 2.6070e+01,
         2.7182e+01, 2.7198e+01, 2.7408e+01, 2.8564e+01, 2.9130e+01, 2.9142e+01,
         2.9179e+01, 2.9179e+01, 2.9179e+01],
        [2.5111e-02, 6.9933e+00, 7.0184e+00, 8.6427e+00, 9.0629e+00, 1.9416e+01,
         2.0549e+01, 2.0935e+01, 2.3965e+01, 2.5659e+01, 2.5659e+01, 3.0627e+01,
         3.1890e+01, 3.1906e+01, 3.2116e+01, 3.3272e+01, 3.3890e+01, 3.3902e+01,
         3.9908e+01, 3.9908e+01, 4.0717e+01],
        [2.5112e-02, 7.0939e+00, 7.1189e+00, 1.3487e+01, 1.3907e+01, 2.4260e+01,
         2.5393e+01, 2.5779e+01, 2.8826e+01, 3.0520e

In [105]:
ss

tensor([[[ 1,  1,  1,  ...,  1,  1,  4],
         [ 1,  1,  1,  ...,  1,  1,  4],
         [ 1,  1,  1,  ...,  1,  1,  4],
         ...,
         [ 1,  1,  1,  ...,  1,  1,  4],
         [ 1,  1,  1,  ...,  1,  1,  4],
         [ 4,  4,  4,  ...,  4,  4, 16]],

        [[ 1,  1,  1,  ...,  1,  1,  6],
         [ 1,  1,  1,  ...,  1,  1,  6],
         [ 1,  1,  1,  ...,  1,  1,  6],
         ...,
         [ 1,  1,  1,  ...,  1,  1,  6],
         [ 1,  1,  1,  ...,  1,  1,  6],
         [ 3,  3,  3,  ...,  3,  3, 18]],

        [[ 1,  1,  1,  ...,  1,  1,  6],
         [ 1,  1,  1,  ...,  1,  1,  6],
         [ 1,  1,  1,  ...,  1,  1,  6],
         ...,
         [ 1,  1,  1,  ...,  1,  1,  6],
         [ 1,  1,  1,  ...,  1,  1,  6],
         [ 3,  3,  3,  ...,  3,  3, 18]],

        ...,

        [[ 8,  8,  8,  ...,  8,  8,  8],
         [ 8,  8,  8,  ...,  8,  8,  8],
         [ 8,  8,  8,  ...,  8,  8,  8],
         ...,
         [ 8,  8,  8,  ...,  8,  8,  8],
         [ 8,  8,  8, 

In [106]:
rois_rescale

tensor([[ 1, 17,  4, 20],
        [15, 13, 17, 18],
        [15, 11, 17, 16],
        [ 7,  7,  8,  8],
        [ 8, 24,  9, 24],
        [12, 10, 14, 11],
        [ 0, 17,  4, 20],
        [ 1, 16,  4, 20],
        [11,  7, 15, 11],
        [18, 15, 21, 19],
        [ 6,  0, 12,  3],
        [ 5, 10,  9, 14],
        [ 0, 10,  9, 19],
        [ 3, 10, 12, 19],
        [ 0,  8,  8, 17],
        [ 8, 11, 17, 19],
        [ 0,  1,  5, 14],
        [10,  8, 17, 16],
        [10, 10, 17, 19],
        [ 7,  7, 17, 25],
        [11, 13, 17, 22],
        [11, 15, 17, 24],
        [ 4, 15, 12, 24],
        [ 2, 11, 11, 20],
        [ 1, 13, 10, 22],
        [ 0,  7,  7, 15],
        [ 7,  0, 16,  7],
        [ 2,  5, 17, 24],
        [ 0,  4, 15, 22],
        [ 3, 10, 17, 25],
        [ 0,  0, 13, 16],
        [ 5,  0, 15, 21],
        [ 3,  0, 14, 24],
        [ 0,  0, 15, 24]])

In [137]:
torch.ones((3, 5, 2, 2))

tensor([[[[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]]],


        [[[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]]],


        [[[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]],

         [[1., 1.],
          [1., 1.]]]])

In [139]:
torch.ones((3, 4)).cumsum(2).cumsum(3)

tensor([[[[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]],

         [[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]],

         [[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]]],


        [[[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]],

         [[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]],

         [[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]]],


        [[[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]],

         [[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]],

         [[ 1.,  2.,  3.,  4.],
          [ 2.,  4.,  6.,  8.],
          [ 3.,  6.,  9., 12.]]]])

In [148]:
(subroi_power_sum < 0).sum()

tensor(13846)

In [149]:
subroi_power_sum.nelement()

426496

In [150]:
subroi_power_sum.size()

torch.Size([34, 7, 7, 256])

In [153]:
a = subroi_power_sum.reshape(-1).detach().numpy().tolist()

In [156]:
res = []
for n in a:
    if n < 0:
        res.append(n)

In [159]:
res

[-4.76837158203125e-07,
 -2.384185791015625e-07,
 -2.384185791015625e-07,
 -2.384185791015625e-07,
 -9.5367431640625e-07,
 -9.5367431640625e-07,
 -3.814697265625e-06,
 -4.76837158203125e-07,
 -9.5367431640625e-07,
 -9.5367431640625e-07,
 -1.9073486328125e-06,
 -4.76837158203125e-07,
 -1.9073486328125e-06,
 -1.430511474609375e-06,
 -9.5367431640625e-07,
 -8.940696716308594e-08,
 -4.76837158203125e-07,
 -4.76837158203125e-07,
 -2.384185791015625e-07,
 -2.384185791015625e-07,
 -2.384185791015625e-07,
 -9.5367431640625e-07,
 -9.5367431640625e-07,
 -3.814697265625e-06,
 -4.76837158203125e-07,
 -9.5367431640625e-07,
 -9.5367431640625e-07,
 -1.9073486328125e-06,
 -4.76837158203125e-07,
 -1.9073486328125e-06,
 -1.430511474609375e-06,
 -9.5367431640625e-07,
 -8.940696716308594e-08,
 -4.76837158203125e-07,
 -4.76837158203125e-07,
 -2.384185791015625e-07,
 -2.384185791015625e-07,
 -2.384185791015625e-07,
 -9.5367431640625e-07,
 -9.5367431640625e-07,
 -3.814697265625e-06,
 -4.76837158203125e-07,
 